In [1]:
from pathlib import Path
import os,sys, json, datetime, getopt
import pickle as pkl
import time
import pandas as pd
import numpy as np
sys.path.append('../Analyses/')
from analyses_table import *

In [3]:
an = 'Ne'
Tasks = ['OF','T3g','T3h','T3j','T3i']

oakPaths = getOakPaths()
aPaths = getAnimalPaths(oakPaths,an)

sessions = []
sePaths = {}
exp = []
nTetrodes=16

if (aPaths['Raw']).exists():
    for session in aPaths['Raw'].glob('*'):
        se = session.name
        try:
            temp = se.split('_')
            anID = temp[0]
            task = temp[1]
            date = temp[2]

            if (anID==an) & (task in Tasks):
                sessions.append(se)
                exp.append(task)
                sePaths[se] = getSessionPaths(oakPaths,se) 
        except:
            pass
        
fn = oakPaths['Root'] / (an+'_SessionPaths.pkl')
with fn.open(mode='wb') as f:
    pkl.dump(sePaths,f,pkl.HIGHEST_PROTOCOL)
 

In [9]:
       
aTable = pd.DataFrame(0,index=sessions,columns=['Task','Raw','PP','PP_A','Sort','Sort_A','Clust','Clust_A','FR','FR_A','Zone','Trial','TrModels','BehavA'])
aTable['Task']=exp

t0=time.time()
aTable = checkRaw(sePaths,aTable)
t1=time.time()
print('Time to check raw files {0:.2f}secs'.format(t1-t0))
aTable = checkPrePro(sePaths,aTable)
t2=time.time()
print('Time to check preprocessed binaries {0:.2f}secs'.format(t2-t1))
aTable = checkSort(sePaths,aTable)
t3=time.time()
print('Time to check sorting {0:.2f}secs'.format(t3-t2))
aTable = checkClust(sePaths,aTable)
t4=time.time()
print('Time to check clusters {0:.2f}secs'.format(t4-t3))
aTable = checkFR(sePaths,aTable)
t5=time.time()
print('Time to check firing rates {0:.2f}secs'.format(t5-t4))
aTable = checkZoneAnalyses(sePaths,aTable)
t6=time.time()
print('Time to check zone analyses {0:.2f}secs'.format(t6-t5))
aTable = checkTrialAnalyses(sePaths,aTable)
t7=time.time()
print('Time to check trial analyses {0:.2f}secs'.format(t7-t6))
print('Completed: Time {0:.2f}'.format(t7-t0))

fn = oakPaths['Root'] / (an+'_AnalysesTable.csv')
aTable.to_csv(fn)

Time to check raw files 432.24secs
Time to check preprocessed binaries 68.83secs
Time to check sorting 67.49secs
Time to check clusters 51.10secs
Time to check firing rates 11.39secs
Time to check zone analyses 2.33secs
Time to check trial analyses 4.28secs
Completed: Time 637.66


dict_keys(['Li_OF_052318', 'Li_OF_052418', 'Li_OF_052518', 'Li_OF_052818', 'Li_OF_052918', 'Li_OF_053018', 'Li_OF_053118', 'Li_OF_060118', 'Li_OF_060418', 'Li_OF_060518', 'Li_OF_060718', 'Li_OF_060818', 'Li_OF_061118', 'Li_OF_061218', 'Li_OF_061318', 'Li_OF_061418', 'Li_OF_061518', 'Li_OF_061818', 'Li_OF_062018', 'Li_OF_062518', 'Li_OF_062618', 'Li_OF_062718', 'Li_OF_062818', 'Li_OF_062918', 'Li_OF_070218', 'Li_OF_070618', 'Li_OF_071018', 'Li_OF_080118', 'Li_OF_080218', 'Li_T3g_052418', 'Li_T3g_052518', 'Li_T3g_052818', 'Li_T3g_052918', 'Li_T3g_053018', 'Li_T3g_053118', 'Li_T3g_060118', 'Li_T3g_060418', 'Li_T3g_061318', 'Li_T3g_061418', 'Li_T3g_061518', 'Li_T3g_061818', 'Li_T3g_062018', 'Li_T3g_062518', 'Li_T3g_062618', 'Li_T3g_062718', 'Li_T3g_062818', 'Li_T3g_062918', 'Li_T3g_070218', 'Li_T3g_070518', 'Li_T3g_070618', 'Li_T3g_070918', 'Li_T3g_071018', 'Li_T3g_071118', 'Li_T3g_072518', 'Li_T3g_072618', 'Li_T3g_073018', 'Li_T3g_080118', 'Li_T3g_080218', 'Li_T3g_080318', 'Li_T3g_080618'

In [23]:
def getOakPaths():
    oakPaths = {}
    oakPaths['Root'] = Path('/mnt/o/giocomo/alexg/')
    oakPaths['Clustered'] = Path('/mnt/o/giocomo/alexg/Clustered/')
    oakPaths['PreProcessed'] = Path('/mnt/o/giocomo/alexg/PreProcessed/')
    oakPaths['Raw'] = Path('/mnt/o/giocomo/alexg/RawData/InVivo/')
    oakPaths['Analyses'] = Path('/mnt/o/giocomo/alexg/Analyses')
    return oakPaths

oakPaths = getOakPaths()

In [33]:
def getAnimalPaths(rootPath,animal):
    Paths = {}
    Paths['Clusters'] = rootPath['Clustered'] / animal
    Paths['Raw'] = rootPath['Raw'] / animal 
    Paths['PreProcessed'] = rootPath['PreProcessed']  
    Paths['Analyses'] = rootPath['Analyses'] / animal
    return Paths

In [47]:
def getSessionPaths(rootPath, session,step=0.02,SR=32000):
    tmp = session.split('_')
    animal = tmp[0]
    task = tmp[1]
    date = tmp[2]

    Paths = {}
    Paths['session'] = session
    Paths['animal']=animal
    Paths['task'] = task
    Paths['date'] = date
    Paths['step'] = step
    Paths['SR'] = SR
    Paths['Clusters'] = rootPath['Clustered'] / animal /(session+'_KSClusters')
    Paths['Raw'] = rootPath['Raw'] / animal / session
    Paths['PreProcessed'] = rootPath['PreProcessed'] / animal / (session + '_Results')
    Paths['Analyses'] = rootPath['Analyses'] / animal/ (session + '_Analyses')

    Paths['ClusterTable'] = rootPath['Clustered'] / animal / (animal+'_ClusteringSummary.json')

    Paths['Analyses'].mkdir(parents=True, exist_ok=True)

    Paths['BehavTrackDat'] = Paths['Analyses'] / ('BehTrackVariables_{}ms.h5'.format(int(step*1000)))

    Paths['Cell_Spikes'] = Paths['Analyses'] / 'Cell_Spikes.json'
    Paths['Cell_Bin_Spikes'] = Paths['Analyses'] / ('Cell_Bin_Spikes_{}ms.npy'.format(int(step*1000)))
    Paths['Cell_FR'] = Paths['Analyses'] / ('Cell_FR_{}ms.npy'.format(int(step*1000)))

    Paths['Mua_Spikes'] = Paths['Analyses'] / 'Mua_Spikes.json'
    Paths['Mua_Bin_Spikes'] = Paths['Analyses'] / ('Mua_Bin_Spikes_{}ms.npy'.format(int(step*1000)))
    Paths['Mua_FR'] = Paths['Analyses'] / ('Mua_FR_{}ms.npy'.format(int(step*1000)))

    Paths['Spike_IDs'] = Paths['Analyses'] / 'Spike_IDs.json'
    Paths['ZoneAnalyses'] = Paths['Analyses'] / 'ZoneAnalyses.pkl'

    Paths['TrialInfo'] = Paths['Analyses'] / 'TrInfo.pkl'
    Paths['TrialCondMat'] = Paths['Analyses'] / 'TrialCondMat.csv'
    Paths['TrLongPosMat'] = Paths['Analyses'] / 'TrLongPosMat.csv'
    Paths['TrLongPosFRDat'] = Paths['Analyses'] / 'TrLongPosFRDat.csv'
    Paths['TrModelFits'] = Paths['Analyses'] /  'TrModelFits.csv'

    # plots directories
    Paths['Plots'] = Paths['Analyses'] / 'Plots'
    Paths['Plots'].mkdir(parents=True, exist_ok=True)
    Paths['SampCountsPlots'] = Paths['Plots'] / 'SampCountsPlots'
    Paths['SampCountsPlots'].mkdir(parents=True, exist_ok=True)

    Paths['ZoneFRPlots'] = Paths['Plots'] / 'ZoneFRPlots'
    Paths['ZoneFRPlots'].mkdir(parents=True, exist_ok=True)

    Paths['ZoneCorrPlots'] = Paths['Plots'] / 'ZoneCorrPlots'
    Paths['ZoneCorrPlots'].mkdir(parents=True, exist_ok=True)
    Paths['SIPlots'] = Paths['Plots'] / 'SIPlots'
    Paths['SIPlots'].mkdir(parents=True, exist_ok=True)

    Paths['TrialPlots'] = Paths['Plots'] / 'TrialPlots'
    Paths['TrialPlots'].mkdir(parents=True, exist_ok=True)
    return Paths


In [34]:
an = 'Li'
Tasks = ['OF','T3g','T3h','T3j','T3i']

aPaths = getAnimalPaths(oakPaths,an)

In [32]:
oakPaths['Clustered']/an

PosixPath('/mnt/o/giocomo/alexg/Clustered/Li')

In [7]:
oakPaths['Raw']/an

PosixPath('/mnt/o/giocomo/alexg/RawData/InVivo/Li')

In [107]:
sessions = []
sePaths = {}
exp = []
nTetrodes=16

if (aPaths['Raw']).exists():
    for session in aPaths['Raw'].glob('*'):
        se = session.name
        try:
            temp = se.split('_')
            anID = temp[0]
            task = temp[1]
            date = temp[2]

            if (anID==an) & (task in Tasks):
                sessions.append(se)
                exp.append(task)
                sePaths[se] = getSessionPaths(oakPaths,se) 
        except:
            pass

In [152]:
def checkRaw(sePaths,aTable):
    for se in aTable.index:
        rawFlag = 1
        for ch in ['a','b','c','d']:
            for tt in np.arange(1,nTetrodes+1):
                if not (sePaths[se]['Raw'] / ('CSC{}{}.ncs'.format(tt,ch))).exists():
                    rawFlag = 0
                    break        
        aTable.loc[se,'Raw']=rawFlag
    return aTable

def checkPrePro(sePaths,aTable):
    for se in aTable.index:
        allTTFlag = 1
        partialFlag = 0 
        for tt in np.arange(1,nTetrodes+1):
            if not (sePaths[se]['PreProcessed'] / ('tt_{}.bin'.format(tt))).exists():
                allTTFlag=0
            else: 
                partialFlag=1
            
        aTable.loc[se,'PP']=partialFlag
        aTable.loc[se,'PP_A']=allTTFlag
    return aTable

def checkSort(sePaths,aTable):
    for se in aTable.index:
        allTTFlag = 1
        partialFlag = 0 
        for tt in np.arange(1,nTetrodes+1):
            if not (sePaths[se]['Clusters'] / ('tt_{}'.format(tt)) / 'rez.mat').exists():
                allTTFlag=0
            else: 
                partialFlag=1
            
        aTable.loc[se,'Sort']=partialFlag
        aTable.loc[se,'Sort_A']=allTTFlag
    return aTable

def checkClust(sePaths,aTable):
    for se in aTable.index:
        allTTFlag = 1
        partialFlag = 0 
        for tt in np.arange(1,nTetrodes+1):
            if not (sePaths[se]['Clusters'] / ('tt_{}'.format(tt)) / 'cluster_group.tsv').exists():
                allTTFlag=0
            else: 
                partialFlag=1
            
        aTable.loc[se,'Clust']=partialFlag
        aTable.loc[se,'Clust_A']=allTTFlag
    return aTable

def checkFR(sePaths,aTable):
    for se in aTable.index:
        allFR = 1
        partialFR = 0
        if not (sePaths[se]['Cell_Bin_Spikes']).exists():
            allFR = 0
        else:
            dat = np.load(sePaths[se]['Cell_Bin_Spikes'])
            if np.all(dat.sum(axis=1)>0):
                partialFR = 1
            else:
                allFR = 0

        aTable.loc[se,'FR']=partialFR
        aTable.loc[se,'FR_A']=allFR
    return aTable

def checkZoneAnalyses(sePaths,aTable):
    for se in aTable.index:
        if not (aTable.loc[se]['Task']=='OF'):
            if sePaths[se]['ZoneAnalyses'].exists():
                aTable.loc[se]['Zone']=1
            else:
                aTable.loc[se]['Zone']=0    
    return aTable

def checkTrialAnalyses(sePaths,aTable):
    for se in aTable.index:
        if not (aTable.loc[se]['Task']=='OF'):
            if sePaths[se]['TrialCondMat'].exists():
                aTable.loc[se]['Trial']= 1
            else:
                aTable.loc[se]['Trial']= 0

            if sePaths[se]['TrModelFits'].exists():
                aTable.loc[se]['TrModels'] = 1
            else:
                aTable.loc[se]['TrModels'] = 0
        
    return aTable

In [136]:
aTable = pd.DataFrame(0,index=sessions,columns=['Task','Raw','PP','PP_A','Sort','Sort_A','Clust','Clust_A','FR','FR_A','Zone','Trial','TrModels','BehavA'])
aTable['Task']=exp

In [137]:
aTable

,Task,Raw,PP,PP_A,Sort,Sort_A,Clust,Clust_A,FR,FR_A,Zone,Trial,TrModels,BehavA
Li_OF_052318,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_052418,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_052518,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_052818,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_052918,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_053018,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_053118,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_060118,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_060418,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_060518,OF,0,0,0,0,0,0,0,0,0,0,0,0,0


In [138]:
t0=time.time()
aTable = checkRaw(sePaths,aTable)
t1=time.time()
print('Time to check raw files {0:.2f}secs'.format(t1-t0))
aTable = checkPrePro(sePaths,aTable)
t2=time.time()
print('Time to check preprocessed binaries {0:.2f}secs'.format(t2-t1))
aTable = checkSort(sePaths,aTable)
t3=time.time()
print('Time to check sorting {0:.2f}secs'.format(t3-t2))
aTable = checkClust(sePaths,aTable)
t4=time.time()
print('Time to check clusters {0:.2f}secs'.format(t4-t3))
aTable = checkFR(sePaths,aTable)
t5=time.time()
print('Time to check firing rates {0:.2f}secs'.format(t5-t4))
aTable = checkZoneAnalyses(sePaths,aTable)
t6=time.time()
print('Time to check zone analyses {0:.2f}secs'.format(t6-t5))
aTable = checkTrialAnalyses(sePaths,aTable)
t7=time.time()
print('Time to check trial analyses {0:.2f}secs'.format(t7-t6))
print('Completed: Time {0:.2f}'.format(t7-t0))

Time to check raw files 130.19secs
Time to check preprocessed binaries 32.64secs
Time to check sorting 34.69secs
Time to check clusters 31.49secs
Time to check firing rates 7.01secs


/home/alexg8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/alexg8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Time to check zone analyses 4.15secs


/home/alexg8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/alexg8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/alexg8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/alexg8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be s

Time to check trial analyses 9.31secs
Completed: Time 249.49


In [153]:
aTable = checkClust(sePaths,aTable)

In [154]:
aTable

,Task,Raw,PP,PP_A,Sort,Sort_A,Clust,Clust_A,FR,FR_A,Zone,Trial,TrModels,BehavA
Li_OF_052318,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_052418,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_052518,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_052818,OF,1,1,1,1,1,1,0,0,0,0,0,0,0
Li_OF_052918,OF,1,1,1,1,1,1,0,0,0,0,0,0,0
Li_OF_053018,OF,1,1,1,1,1,1,0,0,0,0,0,0,0
Li_OF_053118,OF,1,1,1,1,0,1,0,0,0,0,0,0,0
Li_OF_060118,OF,1,1,1,1,1,1,0,0,0,0,0,0,0
Li_OF_060418,OF,1,1,1,1,0,1,0,0,0,0,0,0,0
Li_OF_060518,OF,1,1,1,1,1,1,1,0,0,0,0,0,0


In [156]:
fn = oakPaths['Root'] / (an+'_AnalysesTable.csv')
aTable.to_csv(fn)

fn = oakPaths['Root'] / (an+'_SessionPaths.pkl')
with fn.open(mode='wb') as f:
    pkl.dump(sePaths,f,pkl.HIGHEST_PROTOCOL)


In [145]:
at

,Task,Raw,PP,PP_A,Sort,Sort_A,Clust,Clust_A,FR,FR_A,Zone,Trial,TrModels,BehavA
Li_OF_052318,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_052418,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_052518,OF,0,0,0,0,0,0,0,0,0,0,0,0,0
Li_OF_052818,OF,1,1,1,1,1,1,0,0,0,0,0,0,0
Li_OF_052918,OF,1,1,1,1,1,1,0,0,0,0,0,0,0
Li_OF_053018,OF,1,1,1,1,1,1,0,0,0,0,0,0,0
Li_OF_053118,OF,1,1,1,1,0,1,0,0,0,0,0,0,0
Li_OF_060118,OF,1,1,1,1,1,1,0,0,0,0,0,0,0
Li_OF_060418,OF,1,1,1,1,0,1,0,0,0,0,0,0,0
Li_OF_060518,OF,1,1,1,1,1,1,1,0,0,0,0,0,0
